In [13]:
import requests
from bs4 import BeautifulSoup

def crawl_vnexpress_articles(num_articles=50):
    url = "https://vnexpress.net"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    articles = []
    links = []

    # Lấy các thẻ chứa đường link bài viết chính
    for a in soup.select("a[href^='https://vnexpress.net']"):
        href = a.get("href")
        if href and href not in links and len(links) < num_articles:
            links.append(href)

    print(f"Đã thu thập {len(links)} link bài viết.")

    for link in links:
        try:
            r = requests.get(link, timeout=5)
            s = BeautifulSoup(r.text, "html.parser")

            # Lấy nội dung chính (thẻ article)
            body = s.find("article")
            if body:
                paragraphs = body.find_all("p")
                content = "\n".join([p.get_text() for p in paragraphs])
                if len(content) > 100:
                    articles.append(content)
        except:
            continue

    # Gộp toàn bộ nội dung và lưu
    full_text = "\n".join(articles)
    with open("raw_articles.txt", "w", encoding="utf-8") as f:
        f.write(full_text)

    print(f"Đã lưu {len(articles)} bài viết vào file raw_articles.txt")

# Gọi hàm crawl
crawl_vnexpress_articles()


Đã thu thập 50 link bài viết.
Đã lưu 48 bài viết vào file raw_articles.txt


In [14]:
import re

def clean_and_tokenize(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # 1. Chuyển về chữ thường
    text = text.lower()

    # 2. Loại bỏ số và ký tự đặc biệt (giữ lại chữ cái và khoảng trắng)
    text = re.sub(r'[^a-zàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễ'
                  r'ìíịỉĩòóọỏõôồốộổỗơờớợởỡ'
                  r'ùúụủũưừứựửữỳýỵỷỹđ\s]', '', text)

    # 3. Loại bỏ nhiều khoảng trắng dư thừa
    text = re.sub(r'\s+', ' ', text).strip()

    # 4. Tách từ
    tokens = text.split()

    print(f"Tổng số từ sau khi tách: {len(tokens)}")
    return tokens

# Gọi thử
tokens = clean_and_tokenize('raw_articles.txt')

# (Tùy chọn) Lưu kết quả ra file để xem
with open('tokens.txt', 'w', encoding='utf-8') as f:
    for word in tokens:
        f.write(word + '\n')


Tổng số từ sau khi tách: 30277


In [15]:
from underthesea import word_tokenize

def clean_text(text):
    # Chuyển về chữ thường
    text = text.lower()
    
    # Giữ lại chữ cái tiếng Việt và khoảng trắng
    text = re.sub(r'[^a-zàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễ'
                  r'ìíịỉĩòóọỏõôồốộổỗơờớợởỡ'
                  r'ùúụủũưừứựửữỳýỵỷỹđ\s]', '', text)
    
    # Loại bỏ khoảng trắng dư
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def tokenize_with_underthesea(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        raw_text = f.read()

    cleaned = clean_text(raw_text)

    # Tách từ bằng underthesea (giữ đúng từ ghép)
    tokenized_text = word_tokenize(cleaned, format="text")  # ra dạng: "trí_tuệ nhân_tạo là ..."
    
    # Chuyển về danh sách từ (loại bỏ dấu _ nếu muốn giữ lại dạng "trí tuệ")
    tokens = tokenized_text.replace("_", " ").split()
    
    print(f"Đã tách được {len(tokens)} từ.")
    return tokens

# Gọi hàm
tokens = tokenize_with_underthesea("raw_articles.txt")

# Lưu ra file nếu cần
with open("tokens_underthesea.txt", "w", encoding="utf-8") as f:
    for word in tokens:
        f.write(word + '\n')


Đã tách được 30277 từ.


In [16]:
import os

def load_all_vietnamese_words(folder_path):
    all_words = set()
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    word = line.strip().lower()
                    if word:
                        all_words.add(word)
    return all_words


In [17]:
def filter_vietnamese_words(tokens, vietnamese_dict):
    return [word for word in tokens if word in vietnamese_dict]


In [18]:
# Đường dẫn đến thư mục từ điển
folder_path = 'tu_dien_goc'

# 1. Tải từ điển tiếng Việt từ tất cả file
vietnamese_dict = load_all_vietnamese_words(folder_path)

# 2. Giả sử bạn có tokens từ file tokens_underthesea.txt
with open('tokens_underthesea.txt', 'r', encoding='utf-8') as f:
    tokens = [line.strip().lower() for line in f if line.strip()]

# 3. Lọc từ
vietnamese_only_tokens = filter_vietnamese_words(tokens, vietnamese_dict)

# 4. Loại bỏ trùng lặp bằng set()
unique_tokens = sorted(set(vietnamese_only_tokens))

# 5. Ghi ra file
with open('vietnamese_only_tokens.txt', 'w', encoding='utf-8') as f:
    for word in vietnamese_only_tokens:
        f.write(word + '\n')

print(f"Đã lọc {len(vietnamese_only_tokens)} từ tiếng Việt từ tổng {len(tokens)} từ.")


Đã lọc 28899 từ tiếng Việt từ tổng 30277 từ.


In [19]:
# Đọc file, loại bỏ dòng trống, sắp xếp và ghi lại file

with open("vietnamese_only_tokens.txt", "r", encoding="utf-8") as f:
    words = [line.strip() for line in f if line.strip()]

words_sorted = sorted(words, key=lambda x: x.lower())
unique_sorted = sorted(set(words_sorted), key=lambda x: x.lower())


with open("vietnamese_only_tokens_sorted.txt", "w", encoding="utf-8") as f:
    for word in unique_sorted:
        f.write(word + "\n")

In [20]:
import random
import unicodedata

# Bảng các nguyên âm tiếng Việt và dấu thanh
vowels = "aăâeêioôơuưy"
tone_marks = ["`", "́", "̀", "̉", "̃`", "̣"]  # sắc, huyền, hỏi, ngã, nặng

def random_tone(char):
    # Thêm hoặc thay đổi dấu thanh cho nguyên âm
    if char not in vowels:
        return char
    base = unicodedata.normalize('NFD', char)[0]
    tone = random.choice(tone_marks)
    return unicodedata.normalize('NFC', base + tone)

def generate_typos(word, num_typos=10):
    typo_list = []
    operations = [
        "insert",      # Thêm ký tự ngẫu nhiên
        "delete",      # Thiếu ký tự
        "substitute",  # Thay ký tự ngẫu nhiên
        "transpose",   # Đổi chỗ ký tự
        "repeat",      # Thừa ký tự
        "wrong_tone",  # Sai dấu
    ]
    for _ in range(num_typos):
        op = random.choice(operations)
        typo = None
        if op == "insert" and len(word) > 0:
            pos = random.randint(0, len(word))
            char = chr(random.randint(97, 122))
            typo = word[:pos] + char + word[pos:]
        elif op == "delete" and len(word) > 1:
            pos = random.randint(0, len(word) - 1)
            typo = word[:pos] + word[pos+1:]
        elif op == "substitute" and len(word) > 0:
            pos = random.randint(0, len(word) - 1)
            char = chr(random.randint(97, 122))
            typo = word[:pos] + char + word[pos+1:]
        elif op == "transpose" and len(word) > 1:
            pos = random.randint(0, len(word) - 2)
            typo = word[:pos] + word[pos+1] + word[pos] + word[pos+2:]
        elif op == "repeat" and len(word) > 0:
            pos = random.randint(0, len(word) - 1)
            typo = word[:pos] + word[pos] + word[pos:]  # lặp lại ký tự
        elif op == "wrong_tone" and any(c in vowels for c in word):
            pos = random.choice([i for i, c in enumerate(word) if c in vowels])
            typo = word[:pos] + random_tone(word[pos]) + word[pos+1:]
        # Có thể mở rộng thêm các quy tắc phát âm khác
        if typo and typo != word and typo not in typo_list:
            typo_list.append(typo)
    return typo_list

In [21]:
# Đọc danh sách từ
with open("vietnamese_only_tokens_sorted.txt", "r", encoding="utf-8") as f:
    words = [line.strip() for line in f if line.strip()]

# Sinh lỗi cho từng từ và lưu ra file
with open("typos_for_all_words.txt", "w", encoding="utf-8") as f:
    for word in words:
        typos = generate_typos(word, num_typos=50)  # Số lỗi mỗi từ, có thể thay đổi
        for typo in typos:
            f.write(f"{word}\t{typo}\n")